## Result 1: Slices Equivalence
BANKSY identified two subtypes in mature oligodendrocytes (MOD)  
- These were primarily shown at Bregma = 0.16 and 0.26 in their paper  
- The findings consistently appear across slices in similar regions  

In this project, the main goal is to examine if the two subtypes are truly distinct.   
However, this project is based on transcripts from a slice at Bregma -0.24. It's necessary to show the slices equivalence.  


### data

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd

import sys
import os
sys.path.append(os.path.abspath("../src"))
from utils import load_banksy_result

#### Results of BANKSY

In [ ]:
banksy_folder_path = Path("../data/banksy_results/")

In [ ]:
banksy_result = load_banksy_result(banksy_folder_path/"banksy_cluster.txt")

#### color map

In [ ]:
color_map = {8: 'red', 7: 'orange'}

all_labels = banksy_result['banksy_cluster'].unique()

unique_other_labels = [label for label in all_labels if label not in color_map]
num_other_labels = len(unique_other_labels)

colormap = plt.get_cmap('tab20b', num_other_labels)
other_colors = {label: colormap(i) for i, label in enumerate(unique_other_labels)}

full_color_map = {**color_map, **other_colors}

### All Cell Types
BANKSY applied spatial clustering to all slices using identical parameters.
Here, slices at Bregma = -0.24, 0.16, and 0.26 are compared to assess: 
- whether the same cell types appear across slices
- whether they occupy similar spatial regions

In [ ]:
# Bregma = -0.24
banksy_024 = banksy_result[banksy_result['Bregma'] == -0.24].copy()

banksy_024['x'] = banksy_024['x'] - banksy_024['x'].min()
banksy_024['y'] = banksy_024['y'] - banksy_024['y'].min()

In [ ]:
colors = banksy_024['banksy_cluster'].map(full_color_map)

plt.figure(figsize=(6, 6),  dpi=600)
sc = plt.scatter(
    banksy_024["x"],
    banksy_024["y"],
    s=3,
    c=colors
)

ax = plt.gca()  # Get the current axis
ax.set_aspect('equal', adjustable='box')
# plt.colorbar(sc, shrink=0.8)
plt.xlim(-10, 1810)
plt.ylim(-10, 1810)

# Correct way to hide spines
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

plt.title("Bregma -0.24 mm", fontsize=17)
plt.xlabel("(μm)", fontsize=15)
plt.ylabel("(μm)", fontsize=15)
plt.show()


In [ ]:
# Bregma = 0.16
banksy_016 = banksy_result[banksy_result['Bregma'] == 0.16].copy()

banksy_016['x'] = banksy_016['x'] - banksy_016['x'].min()
banksy_016['y'] = banksy_016['y'] - banksy_016['y'].min()

In [ ]:
colors = banksy_016['banksy_cluster'].map(full_color_map)

plt.figure(figsize=(6, 6),  dpi=600)
sc = plt.scatter(
    banksy_016["x"],
    banksy_016["y"],
    s=3,
    c=colors
)

ax = plt.gca()  # Get the current axis
ax.set_aspect('equal', adjustable='box')
plt.xlim(-10, 1810)
plt.ylim(-10, 1810)

ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

plt.title("Bregma 0.16 mm")
plt.xlabel("(μm)")
plt.ylabel("(μm)")
plt.show()

In [ ]:
# Bregma = 0.26
banksy_026 = banksy_result[banksy_result['Bregma'] == 0.26].copy()

banksy_026['x'] = banksy_026['x'] - banksy_026['x'].min()
banksy_026['y'] = banksy_026['y'] - banksy_026['y'].min()

In [ ]:
colors = banksy_026['banksy_cluster'].map(full_color_map)

plt.figure(figsize=(6, 6),  dpi=600)
sc = plt.scatter(
    banksy_026["x"],
    banksy_026["y"],
    s=3,
    c=colors
)

ax = plt.gca()  # Get the current axis
ax.set_aspect('equal', adjustable='box')
plt.xlim(-10, 1810)
plt.ylim(-10, 1810)

# Correct way to hide spines
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

plt.title("Bregma 0.26 mm", fontsize = 13)
plt.xlabel("(μm)")
plt.ylabel("(μm)")
plt.show()


### Stacked Bar Plot
Used to compare cell type proportions across slices, as part of the equivalence analysis.

In [ ]:
banksy_barplot = banksy_result[(banksy_result['Bregma'] == 0.26) | (banksy_result['Bregma'] == 0.16) | (banksy_result['Bregma'] == -0.24)].copy()

In [ ]:
color_map = {8: 'red', 7: 'orange'}

all_labels = banksy_barplot['banksy_cluster'].unique()
unique_other_labels = [label for label in all_labels if label not in color_map]
num_other_labels = len(unique_other_labels)

colormap = plt.get_cmap('tab20b', num_other_labels)
other_colors = {label: colormap(i) for i, label in enumerate(unique_other_labels)}
full_color_map = {**color_map, **other_colors}

df_counts = banksy_barplot.groupby(["Bregma", "banksy_cluster"]).size().unstack(fill_value=0)
df_normalized = df_counts.div(df_counts.sum(axis=1), axis=0)
df_normalized = df_normalized.sort_index(ascending=False)


plt.figure(figsize=(15, 6), dpi=600)

ax = df_normalized.plot(kind="barh", stacked=True, figsize=(15, 6), color=[full_color_map.get(label) for label in df_normalized.columns])

ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.get_legend().remove()
ax.tick_params(axis='x', labelsize=11)
ax.tick_params(axis='y', labelsize=11)

plt.xlabel("Clusters Proportion",fontsize=12)
plt.ylabel("Bregma",fontsize=12)
plt.title("Normalized Stacked Bar Plot of BANKSY Cluster by Bregma",fontsize=13)

plt.show()


### UMAP
UMAP embeddings from BANKSY is used to visually compare clustering consistency across slices.  

In [ ]:
all_UMAP = pd.read_csv(banksy_folder_path / 'umap.csv', sep = ',')
all_UMAP.loc[:, 'banksy'] = banksy_result['banksy_cluster'].values

In [ ]:
colors = all_UMAP['banksy'].map(full_color_map)

plt.figure(figsize=(8, 8),  dpi=600)

sc = plt.scatter(all_UMAP['UMAP_1'], all_UMAP['UMAP_2'], s=3, c=colors)

ax = plt.gca()
ax.set_aspect('equal', adjustable='box')

ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

plt.title("All Slices", fontsize=17)
plt.xlabel("UMAP1", fontsize=17)
plt.ylabel("UMAP2", fontsize=17)
plt.xlim(-20,30)
plt.ylim(-30,20)
plt.show()

In [ ]:
B24_ids = pd.read_csv(banksy_folder_path / 'umap_B24.csv', sep = ',')
B24_ids['Cell_id'] = "Animal_1_" + B24_ids['cell_ids']
B24_UMAP = all_UMAP[all_UMAP['Cell_id'].isin(B24_ids['Cell_id'])]

In [ ]:
colors = B24_UMAP['banksy'].map(full_color_map)

plt.figure(figsize=(8, 8),  dpi=600)

sc = plt.scatter(B24_UMAP['UMAP_1'], B24_UMAP['UMAP_2'], s=3, c=colors)

ax = plt.gca()
ax.set_aspect('equal', adjustable='box')

ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

plt.title("Bregma -0.24 mm", fontsize=17)
plt.xlabel("UMAP1", fontsize=17)
plt.ylabel("UMAP2", fontsize=17)
plt.xlim(-20,30)
plt.ylim(-30,20)
plt.show()

In [ ]:
B16_UMAP = pd.read_csv(banksy_folder_path / 'umap_B16.csv', sep = ',')
B16_UMAP.loc[:, 'banksy'] = banksy_016['banksy_cluster'].values

In [ ]:
colors = B16_UMAP['banksy'].map(full_color_map)

plt.figure(figsize=(8, 8),  dpi=600)

sc = plt.scatter(B16_UMAP['UMAP_1'], B16_UMAP['UMAP_2'], s=3, c=colors)

ax = plt.gca()
ax.set_aspect('equal', adjustable='box')

ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

plt.title("Bregma 0.16 mm", fontsize=17)
plt.xlabel("UMAP1", fontsize=17)
plt.ylabel("UMAP2", fontsize=17)
plt.xlim(-20,30)
plt.ylim(-30,20)
plt.show()

In [ ]:
B26_UMAP = pd.read_csv(banksy_folder_path / 'umap_B26.csv', sep = ',')
B26_UMAP.loc[:, 'banksy'] = banksy_026['banksy_cluster'].values

In [ ]:
colors = B26_UMAP['banksy'].map(full_color_map)

plt.figure(figsize=(8, 8),  dpi=600)

sc = plt.scatter(B26_UMAP['UMAP_1'], B26_UMAP['UMAP_2'], s=3, c=colors)

ax = plt.gca()
ax.set_aspect('equal', adjustable='box')

ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

plt.title("Bregma 0.26 mm", fontsize=17)
plt.xlabel("UMAP1", fontsize=17)
plt.ylabel("UMAP2", fontsize=17)
plt.xlim(-20,30)
plt.ylim(-30,20)
plt.show()

### MOD Distribution
MOD subtypes are identified in all slices and localize to similar regions, supporting spatial and compositional equivalence across slices.


In [ ]:
banksy_024_MOD = banksy_024[(banksy_024['banksy_cluster'] == 7) | (banksy_024['banksy_cluster'] == 8)]
banksy_024_MOD = banksy_024_MOD.copy()

In [ ]:
color_map = {7: 'orange', 8: 'red'}
colors = banksy_024_MOD['banksy_cluster'].map(color_map)

plt.figure(figsize=(6, 6),  dpi=600)
sc = plt.scatter(
    banksy_024_MOD["x"],
    banksy_024_MOD["y"],
    s=3,
    c=colors,
)

ax = plt.gca()
ax.set_aspect('equal', adjustable='box')
plt.xlim(-10, 1810)
plt.ylim(-10, 1810)

ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

plt.title("Bregma -0.24 mm")
plt.xlabel("(μm)")
plt.ylabel("(μm)")
plt.show()


In [ ]:
banksy_016_MOD = banksy_016[(banksy_016['banksy_cluster'] == 7) | (banksy_016['banksy_cluster'] == 8)].copy()
banksy_016_MOD['x'] = banksy_016_MOD['x'] - banksy_016_MOD['x'].min()
banksy_016_MOD['y'] = banksy_016_MOD['y'] - banksy_016_MOD['y'].min()
banksy_016_MOD = banksy_016_MOD.copy()

In [ ]:
color_map = {7: 'orange', 8: 'red'}
colors = banksy_016_MOD['banksy_cluster'].map(color_map)

plt.figure(figsize=(6, 6),  dpi=600)
sc = plt.scatter(
    banksy_016_MOD["x"],
    banksy_016_MOD["y"],
    s=3,
    c=colors,
)

ax = plt.gca()
ax.set_aspect('equal', adjustable='box')
plt.xlim(-10, 1810)
plt.ylim(-10, 1810)

ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

plt.title("Bregma 0.16 mm")
plt.xlabel("(μm)")
plt.ylabel("(μm)")
plt.show()


In [ ]:
banksy_026_MOD = banksy_026[(banksy_026['banksy_cluster'] == 7) | (banksy_026['banksy_cluster'] == 8)].copy()
banksy_026_MOD['x'] = banksy_026_MOD['x'] - banksy_026_MOD['x'].min()
banksy_026_MOD['y'] = banksy_026_MOD['y'] - banksy_026_MOD['y'].min()
banksy_026_MOD = banksy_026_MOD.copy()

In [ ]:
color_map = {7: 'orange', 8: 'red'}
colors = banksy_026_MOD['banksy_cluster'].map(color_map)

plt.figure(figsize=(6, 6),  dpi=600)
sc = plt.scatter(
    banksy_026_MOD["x"],
    banksy_026_MOD["y"],
    s=3,
    c=colors,
)

ax = plt.gca()
ax.set_aspect('equal', adjustable='box')
plt.xlim(-10, 1810)
plt.ylim(-10, 1810)

ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

plt.title("Bregma 0.26 mm")
plt.xlabel("(μm)")
plt.ylabel("(μm)")
plt.show()
